In [4]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import model_selection
from sklearn import linear_model
from sklearn import preprocessing

In [5]:
data = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# Define function to extract titles from passenger names
def get_title(name):
            title_search = re.search(' ([A-Za-z]+)\.', name)
            # If the title exists, extract and return it.
            if title_search:
                return title_search.group(1)
            return ""

In [ ]:
def preproc_data(XX, cc, nc):

    if('Name') in XX.columns:
    
        
        # Create a new feature Title, containing the titles of passenger names
        XX.loc[:,'Name'] = XX['Name'].apply(get_title)
        # Group all non-common titles into one single grouping "Rare"
        XX.loc[:,'Name'] = XX['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                                     'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

        XX.loc[:,'Name'] = XX['Name'].replace('Mlle', 'Miss')
        XX.loc[:,'Name'] = XX['Name'].replace('Ms', 'Miss')
        XX.loc[:,'Name'] = XX['Name'].replace('Mme', 'Mrs')
    
    if('Age') in XX.columns:
        age = XX[['Pclass','Age']].groupby(by='Pclass').median()
        XX.loc[:,'Age'] = [age.loc[XX.Pclass[i]].values[0] if pd.isna(XX.Age[i]) else XX.Age[i] 
                    for i in XX[['Pclass', 'Age']].index]
        ## create bin for age features
        #XX.loc[:,'Age_bin'] = pd.cut(XX['Age'], bins=[0,12,20,40,120], labels=['Children','Teenage','Adult','Elder'])
        #cc.append('Age_bin')
        #nc.remove('Age')

    if('Fare') in XX.columns:
        XX.loc[:,'Fare'].fillna(XX['Fare'].mean(),inplace=True)
        
        ## create bin for fare features
        ##XX.loc[:,'Fare_bin'] = pd.cut(XX['Fare'], bins=[0,7.91,14.45,31,120], 
                               #labels=['Low_fare','median_fare','Average_fare','high_fare'])
        ##cc.append('Fare_bin')
        ##nc.remove('Fare')
        
    
    if('Embarked') in XX.columns:
        emb = pd.Series(Counter(XX.Embarked))
        XX.loc[:,'Embarked'].fillna(emb.idxmax(),inplace=True)

    
    if ('SibSp' in  XX.columns) & ('Parch' in  XX.columns):
        XX.loc[:,'FamilySize'] = XX['SibSp'] + XX['Parch'] + 1
        nc.append('FamilySize')
        
    if ('Cabin') in XX.columns:
        
    
    if(len(cc)>0):
        cat = pd.get_dummies(XX[cc])
        return pd.concat((XX[nc],cat),axis=1)
    else:
        return XX[nc]